In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
# 学习速率
learning_rate = 0.0005
# 迭代次数
num_epoch = 3
# 批量大小
batch_size = 128
# 图片大小
img_width = 28
img_height = 28
# 分类数
num_classes = 10

In [3]:
# x，y占位
X = tf.placeholder(tf.float32, [None, img_height, img_width])
Y = tf.placeholder(tf.float32, [None, num_classes])

x_input = tf.reshape(X, [-1, img_height, img_width, 1])

In [4]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='VALID')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')

In [5]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1, dtype=tf.float32)
    return tf.Variable(initial, dtype=tf.float32)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape, dtype=tf.float32)
    return tf.Variable(initial, dtype=tf.float32)

In [6]:
# 在5*5的patch中计算出32个特征
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [7]:
h_conv1 = tf.nn.relu(tf.nn.bias_add(conv2d(x_input, W_conv1), b_conv1))
print(h_conv1)
h_pool1 = max_pool_2x2(h_conv1)
print(h_pool1)

Tensor("Relu:0", shape=(?, 24, 24, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 12, 12, 32), dtype=float32)


In [8]:
# 全连接层，将整个图片的特征，由6272个特征，缩小到1024个特征
W_fc1 = weight_variable([12 * 12 * 32, 1024])
b_fc1 = bias_variable([1024])

h_pool_flat = tf.reshape(h_pool1, [-1, 12*12*32])
h_fc1 = tf.nn.relu(tf.matmul(h_pool_flat, W_fc1) + b_fc1)

In [9]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

print(h_fc1_drop)

Tensor("dropout/mul:0", shape=(?, 1024), dtype=float32)


In [10]:
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

logits = tf.matmul(h_fc1_drop, W_fc2) + b_fc2
y_predict=tf.nn.softmax(logits)
print(y_predict)

Tensor("Softmax:0", shape=(?, 10), dtype=float32)


In [12]:
# 损失函数
tf.nn.sparse_softmax_cross_entropy_with_logits
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
# 训练方法
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss_op)
# 初始化
init = tf.global_variables_initializer()

In [13]:
# argmax最大值index，axis=0纵向，axis=1横向
correct_predict = tf.equal(tf.argmax(y_predict, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

In [14]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
print(x_train.shape)

(60000, 28, 28)


In [15]:
x_train = x_train / 255
x_test = x_test / 255

In [16]:
num_examples = x_train.shape[0]
total_batch = int(num_examples/batch_size)

In [17]:
# One-Hot编码
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [18]:
def next_batch(i):
    a = i * batch_size
    b = (i+1) * batch_size
    return (x_train[a:b], y_train[a:b])

In [19]:
import numpy as np
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(num_epoch):
        avg_cost = 0.
        avg_acc = 0.
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(i)
            sess.run(train_op, feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.5})
            c, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 1.0})
            # Compute average loss
            avg_cost += c / total_batch
            avg_acc += acc / total_batch
        print("Epoch:", '%04d' % (epoch+1), "loss=", "{:.9f}".format(avg_cost), "accuracy=", "{:.9f}".format(avg_acc))
    print("Optimization Finished!")

Epoch: 0001 loss= 0.277917082 accuracy= 0.920255743
Epoch: 0002 loss= 0.087215526 accuracy= 0.974108574
Epoch: 0003 loss= 0.052230373 accuracy= 0.984391693
Optimization Finished!
